### Connect to gdrive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/MyDrive

### Install mimikit

In [ ]:
!pip install git+https://github.com/ktonal/mimikit@experiment/new-data-again

### cd into project's repo

In [ ]:
%cd AIMC-Submission-2021

## Imports and setup neptune

In [ ]:
import matplotlib.pyplot as plt

from mimikit.models.freqnet import FreqNet
from mimikit.models.sample_rnn import SampleRNN
from mimikit import audio

# code is in the repo
from project_utils import *



# where we download states and build dbs :

root_dir = "/content/"

## What's in the project

In [ ]:
project_df = get_project_summary()
project_df

## download list of ids

In [ ]:
my_exp_ids = []

# or download them all :
# my_exp_ids = project_df["id"]

download_states(root_dir, *my_exp_ids)

## List of downloaded checkpoints (exp_id, epoch)

In [ ]:
all_checkpoints = list_downloaded_checkpoints(root_dir)

all_checkpoints

# Generate

In [ ]:
my_checkpoints = [("EXS-47", 75), ("EXS-55", 25)]

# or generate from them all !

# my_checkpoints = all_checkpoints

n_prompts = 4
n_steps = 1000
temperature = 1.

for ckpt in my_checkpoints:
    
    model = load_exp_id_at_epochs(root_dir, ckpt)[0]
    
    print()
    print("******** OUTPUT FOR", ckpt[0], "epoch =", ckpt[1], " **************")
    print()
    
    if not isinstance(model, FreqNet):
        prompt = model.warm_up()[:n_prompts]
    else:
        prompt = model.random_train_batch()[0][:n_prompts]
        
    outpt = model.generate(prompt, n_steps, decode_outputs=True, temperature=temperature)
    
    for i in range(outpt.size(0)):
        y = outpt[i].squeeze().cpu().numpy()
        plt.figure(figsize=(20, 3))
        plt.plot(y)
        plt.show()
        audio(y, sr=model.hparams.get("sr", 22050), hop_length=model.hparams.get("hop_length", 512))
        